In [1]:
from IPython.display import Markdown, display

In [2]:
# install package
# ! pip install -U langchain-ollama

In [13]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama

In [20]:
deepseek_32b = ChatOllama(model="deepseek-r1:32b", request_timeout=360.0, temperature=0.2)
deepseek_8b = OllamaLLM(model="deepseek-r1:8b-llama-distill-fp16", request_timeout=360.0, temperature=0.2)
deepseek_14b = ChatOllama(model="deepseek-r1:14b-qwen-distill-fp16", request_timeout=360.0, temperature=0.2)

In [ ]:
from langchain_core.prompts import PromptTemplate

reasoning_prompt_str = """
You are a football scouting and analytics assistant working for a professional football team in NFL.
Given the abbreviation for positions and teams:
1. Positions: QB, RB, FB, WR, TE, OT, OG, C, DT, DE, LB, CB, DB, S, K, P;
2. Teams: "LA" is the Rams, "LAC" are the Chargers. ARZ, ATL, BAL, BUF, CAR, CHI, CIN, CLE, DAL, DEN, DET, GB, HOU, IND, JAC, KC, LA, LAC, MIA, MIN, NE, NO, NYG, NYJ, LV, PHI, PIT, SEA, SF, TB, TEN, WAS;
Please write down a list of sub questions to help the scouts and analysts to get all information when your football general manager ask "{query_str}".  

You must follow these strict rules:
1. If the question is complicated please break it into several sub questions. 
2. Output ONLY the questions without any other texts, explanations or answers.
3. Output questions MUST contain specific teams name and player's position given previously.
4. Use numbered list for each question.
5. Mention about the team, the position, the date in each question.

Answer: Let's think step by step.
"""
reasoning_prompt = PromptTemplate(input_variables="query_str", template=reasoning_prompt_str)

In [27]:
chain = reasoning_prompt | deepseek_32b

In [32]:
query = "What's going on with Baltimore Ravens special team in December 2024?"
result = chain.invoke({"query_str": query})

In [33]:
result.content

"<think>\nOkay, so I need to figure out what's going on with the Baltimore Ravens' special teams in December 2024. Hmm, where do I start? Well, first off, I should probably break this down into smaller parts because it's a pretty broad question. Let me think about all the components that make up a team's special teams unit.\n\nAlright, so special teams usually involve kickers, punters, returners, and coverage units. Each of these areas can have their own issues or strengths. I should probably look at each position individually to get a comprehensive view.\n\nStarting with the kicker. The Ravens' kicker is important because they handle field goals and extra points. I wonder how reliable he's been in December 2024. Has he been making his kicks, especially in crucial moments? Also, has there been any consistency in his performance?\n\nNext up, the punter. Punting is all about pinning the opponent deep in their territory. So, I should check if the punter has been effective. How's his avera

In [34]:
import re

# Step 1: Extract text after the </think> tag.
marker = "</think>"
idx = result.content.find(marker)
after_think = result.content[idx + len(marker):].strip() if idx != -1 else result.content

# Step 2: Use a regex pattern to extract numbered questions.
pattern = r'\d+\.\s*(.*?)\s*(?=\d+\.|$)'
questions = re.findall(pattern, after_think, flags=re.DOTALL)

# Step 3: Print out each question.
for i, q in enumerate(questions, start=1):
    print(f"q{i} = \"{q.strip()}\"")

q1 = "How has the Baltimore Ravens kicker performed in terms of field goal accuracy and consistency during December 2024?"
q2 = "What is the performance of the Baltimore Ravens punter, including average yards per punt and consistency, in December 2024?"
q3 = "How have the kick returners for the Baltimore Ravens been performing in December 2024, particularly in terms of explosiveness and reliability?"
q4 = "What about the punt returners for the Baltimore Ravens? Have they been effective in generating big plays or contributing to scoring drives this month?"
q5 = "Are there any injury concerns affecting the special teams unit of the Baltimore Ravens as of December 2024?"
q6 = "How has the coaching staff, particularly the special teams coach, been handling the unit's performance and strategy during December 2024?"
q7 = "Have the Baltimore Ravens faced any specific challenges from opposing teams' strategies targeting their special teams in December 2024?"
q8 = "What have been the key perfor